In [1]:
import pandas as pd
import numpy as np
import glob
import random
import pickle
import copy

In [2]:
gamma = 0.99
n_memory = 1000000
explor = False
fit_count = 0
best_advanced_step=0
bestofbest_step = 0
exploration_rate =0

In [3]:
temp = r'input_data\*csv'
csv_dir = glob.glob(temp)
for i in range(len(csv_dir)):
    print(i, csv_dir[i])

0 인풋데이터\Input_Data_ID0_Step1_Value_3\out10.csv
1 인풋데이터\Input_Data_ID0_Step1_Value_3\out11.csv
2 인풋데이터\Input_Data_ID0_Step1_Value_3\out12.csv
3 인풋데이터\Input_Data_ID0_Step1_Value_3\out13.csv
4 인풋데이터\Input_Data_ID0_Step1_Value_3\out14.csv
5 인풋데이터\Input_Data_ID0_Step1_Value_3\out15.csv
6 인풋데이터\Input_Data_ID0_Step1_Value_3\out16.csv
7 인풋데이터\Input_Data_ID0_Step1_Value_3\out17.csv
8 인풋데이터\Input_Data_ID0_Step1_Value_3\out18.csv
9 인풋데이터\Input_Data_ID0_Step1_Value_3\out19.csv
10 인풋데이터\Input_Data_ID0_Step1_Value_3\out2.csv
11 인풋데이터\Input_Data_ID0_Step1_Value_3\out20.csv
12 인풋데이터\Input_Data_ID0_Step1_Value_3\out21.csv
13 인풋데이터\Input_Data_ID0_Step1_Value_3\out22.csv
14 인풋데이터\Input_Data_ID0_Step1_Value_3\out23.csv
15 인풋데이터\Input_Data_ID0_Step1_Value_3\out24.csv
16 인풋데이터\Input_Data_ID0_Step1_Value_3\out25.csv
17 인풋데이터\Input_Data_ID0_Step1_Value_3\out26.csv
18 인풋데이터\Input_Data_ID0_Step1_Value_3\out27.csv
19 인풋데이터\Input_Data_ID0_Step1_Value_3\out28.csv
20 인풋데이터\Input_Data_ID0_Step1_Value_3\out29.csv
21 

In [4]:
# df_list = []
# for i in range(len(csv_dir)):
#     data = pd.read_csv(csv_dir[i], header = None)
#     tmp = data.iloc[:,0].unique()
#     for j in range(len(tmp)):
#         temp = data[data[0]==tmp[j]].set_index(1)
#         temp.columns = ['prod_name', 'z_axis', 'CentSFR 0.7']
#         df_list.append(temp)

In [5]:
# with open('df_list.pickle', 'wb') as f:
#     pickle.dump(df_list, f)
with open('df_list.pickle', 'rb') as f:
    df_list = pickle.load(f)

In [6]:
df_list[50000]

,prod_name,z_axis,CentSFR 0.7
1,,,
0,20223M133784,-10.350,0.105402
1,20223M133784,-10.375,0.112794
2,20223M133784,-10.400,0.238004
3,20223M133784,-10.410,0.374730
4,20223M133784,-10.420,0.517354
5,20223M133784,-10.430,0.654284
6,20223M133784,-10.440,0.766055
7,20223M133784,-10.450,0.833290
8,20223M133784,-10.460,0.845576


In [7]:
len(df_list)

52962

In [8]:
train_test_ratio = 0.7
idx = np.arange(0,len(df_list))
random.shuffle(idx)
train_df_list=[]
test_df_list=[]
for i in range(int(len(df_list)*train_test_ratio)):
    train_df_list.append(df_list[idx[i]])
for i in range(int(len(df_list)*train_test_ratio),len(df_list)):
    test_df_list.append(df_list[idx[i]])

In [9]:
def decide(model, cur_state):
    global explor
    if random.random()>exploration_rate:
        tmp = model.predict(cur_state)
        action = tmp.argmax()+1
        explor = False
    else:
        action = random.randint(1,5)
        explor = True
    return action

In [10]:
def best_decide(model, cur_state):


    tmp = model.predict(cur_state)
    action = tmp.argmax()+1
    explor = False

    return action

In [11]:
def random_load(df_list):
    i=1
    while i!=0:
        tmp = df_list[random.randint(0,len(df_list)-1)]
        i = tmp.index[0]
#     tmp = tmp.loc[tmp.fillna(0)['CentSFR 0.7']!=0,:].iloc[:-3,:]

#     if len(tmp)>9:
#         step = tmp.iloc[2:,2].argmax()+2
    if tmp['CentSFR 0.7'].argmax() == 0:
        step = tmp.iloc[2:,2].argmax()+2
    else:
        step = tmp['CentSFR 0.7'].argmax()
    
    value = tmp.iloc[step,:]['CentSFR 0.7']
    return [tmp, step, value]

In [12]:
def select_load(df_list,k):
    i=1
    while i!=0:
        tmp = df_list[k]
        i = tmp.index[0]
#     tmp = tmp.loc[tmp.fillna(0)['CentSFR 0.7']!=0,:].iloc[:-3,:]
#     if len(tmp)>9:
#         step = tmp.iloc[2:,2].argmax()+2
    if tmp['CentSFR 0.7'].argmax() == 0:
        step = tmp.iloc[2:,2].argmax()+2
    else:
        step = tmp['CentSFR 0.7'].argmax()
    
    value = tmp.iloc[step,:]['CentSFR 0.7']
    return [tmp, step, value]

In [13]:
def init_env(data):
    return [0, data[0].iloc[0,2], data[1]]

In [14]:
# cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
def update_env(data,cur_env,action,memory_D, cur_state):
    global terminal, fit_count, succeeded, failed
    tmp=[]
    tmp.append(cur_state.copy())
    tmp.append(action)
    
    cur_env[0] += action # 누적 스탭
    if cur_env[0] >= len(cur_data[0].index):
        cur_env[1] = data[0].iloc[len(cur_data[0].index)-1,2].copy()
    else:
        cur_env[1] = data[0].iloc[cur_env[0],2].copy() # 현재 SFR값
    cur_state[0][1] = cur_state[0][0].copy() # 바로 전 z 거리
    cur_state[0][3] = cur_state[0][2].copy() # 바로 전 SFR 값
    if cur_env[0] >= len(cur_data[0].index):
        cur_state[0][0] = data[0].iloc[len(cur_data[0].index)-1,1].copy()
    else:
        cur_state[0][0] = data[0].iloc[cur_env[0],1].copy() # 현재 z 거리
    cur_state[0][2] = cur_env[1].copy() # 현재 SFR 값
    cur_state[0][4] = (cur_state[0][2]-cur_state[0][3])/(cur_state[0][1] - cur_state[0][0]) # 기울기 (reward)
    
    reward = 0
    if cur_env[0] > cur_env[2]:
        terminal = True
        reward = -1
#         print('failed')
#         print('exploration : ', explor)
        if explor ==False:
            failed += 1
    if cur_env[0] == cur_env[2]:
        terminal = True
        reward = 1
#         print('succeeded')
        succeeded += 1
    if cur_env[0] < cur_env[2]:
        reward = cur_state[0][2]-cur_state[0][3]
    tmp.append(reward)
    tmp.append(cur_state.copy())
    tmp.append(terminal)
    
    memory_D.append(tmp)
    if len(memory_D)>n_memory:
        memory_D.pop(0)
    fit_count += 1
        
    return cur_env, cur_state, reward, memory_D

In [15]:
def model_fit(model, model2, memory_D):
    if len(memory_D)>100:
        train_x=[]
        delt = []
        train_y=[]
        for i in range(30):
            temp = memory_D[random.randint(0,len(memory_D)-1)]
            tmp = np.zeros(5)
            tmp[temp[1]-1]=1.0

            if temp[4] == False:
                fu_reward = gamma*model2.predict(temp[3]).max()
                delt.append(temp[2]+fu_reward)
            elif temp[4] == True:

                delt.append(temp[2])
            train_x.append(copy.deepcopy(temp[0][0]))
            train_y.append(copy.deepcopy(tmp))
        train_y = np.asarray(train_y)
        train_x = [np.asarray(train_x),np.asarray(delt), train_y]
        
        train_model.fit(train_x, train_y,verbose=0)

    return model

In [16]:
# def model_fit(model, model2, memory_D):
#     if len(memory_D)>100:
#         train_x=[]
#         delt = []
#         train_y=[]
#         for i in range(30):
#             temp = memory_D[random.randint(0,len(memory_D)-1)]
#             tmp = model.predict(temp[0])
#             if temp[4] == False:
#                 fu_reward = gamma*model2.predict(temp[3]).max()
#                 tmp[0][temp[1]-1] = temp[2]+fu_reward
#             elif temp[4] == True:

#                 tmp[0][temp[1]-1] = temp[2]
#             train_x.append(copy.deepcopy(temp[0][0]))
#             train_y.append(copy.deepcopy(tmp[0]))
#         train_x = np.asarray(train_x)
#         train_y = np.asarray(train_y)
        
#         model.fit(train_x, train_y,verbose=0)

#     return model

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Input, Lambda
from tensorflow.keras.optimizers import Adam
def make_model():
    def custom_loss(y_true, y_pred, delta):
        out =  tf.math.square(tf.math.multiply(tf.math.subtract(y_pred, delta), y_true))
        return tf.math.reduce_mean(tf.math.reduce_sum(out, axis=1))
    delta = Input(shape=(1,),dtype= np.float32)
    y_true = Input(shape=(5,))
    x1 = Input(shape=(5,),dtype= np.float32)
    x = Dense(500,activation='relu')(x1)
    x = Dense(200,activation='relu')(x)
    x = Dense(100,activation='relu')(x)
    #     fc1 = Dense(50)(x)
    advantage = Dense(5,activation='linear')(x)
    #     fc2 = Dense(50)(x)
    value = Dense(1)(x)
    policy = Lambda(lambda x: tf.math.add(tf.math.subtract(x[0],tf.math.reduce_mean(x[0], 1, keepdims=True)),x[1]))([advantage, value])
    model = Model([x1],[policy])
    model.compile(optimizer = 'rmsprop', loss='mse')
    train_model = Model([x1, delta, y_true],[policy])
    train_model.add_loss(custom_loss(y_true, policy, delta))
    train_model.compile(optimizer = Adam(learning_rate=0.00005), loss=None)
    return model, train_model
model, train_model = make_model()
model2, train_model2 = make_model()

train_model.summary()

Using TensorFlow backend.


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 500)          3000        input_2[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200)          100200      dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 100)          20100       dense_2[0][0]                    
____________________________________________________________________________________________

In [18]:
train_x = [np.array([[1,1,1,1,1],[1,1,1,1,1]]),np.array([[1],[1]])]
train_y = [np.array([[0,0,0,1.0,0],[0,1.0,0,0,0]])]

In [19]:
train_x = [np.array([[1,1,1,1,1],[1,1,1,1,1]]),np.array([[1],[1]])]
train_y = [np.array([[0,0,0,1,0],[0,1,0,0,0]])]
# train_model.fit(train_x,train_y,verbose=1)
model.predict([np.array([[1,1,1,1,1],[1,1,1,1,1]])])

Epoch 1/1
2/2 [==============================] - 0s 142ms/step - loss: 2.0034


array([[0.23689121, 0.63764954, 0.5936514 , 0.74322355, 0.51885885],
       [0.23689121, 0.63764954, 0.5936514 , 0.74322355, 0.51885885]],
      dtype=float32)

In [20]:
from tensorflow.keras.models import load_model
# main
cur_data = random_load(train_df_list)
cur_env = init_env(cur_data) # cur_step, cur_SFR, terminal_step
# model.fit(np.asarray([np.ones(5)]),np.asarray([np.ones(5)]),verbose=0)

# model = load_model('best_model(0).h5')
# model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
model2.set_weights(model.get_weights())

memory_D = []
best_acc_reward = 0
acc_step=0
acc_reward = 0
advanced_step = 0
succeeded = 0
failed = 0
action_count = 0
# with open('memory_D(1).pickle', 'rb') as f:
#     memory_D = pickle.load(f)
for j in range(1000000):
#     print(j)

#     acc_reward=0
    

    acc_step += 1
    cur_data = random_load(train_df_list)
    cur_env = init_env(cur_data)
    terminal = False
    # cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
    cur_state = np.expand_dims(np.asarray([cur_data[0].iloc[0,1], cur_data[0].iloc[0,1], cur_env[1], cur_env[1],0]),axis=0)
    reward = 0
    cur_step = 0
    episode_reward = 0
    episode_step = 0
    while terminal == False:
        action = decide(model, cur_state)
        cur_step += action
#             print(cur_step, end = ' ')
        past_state = cur_state
        cur_env, cur_state, reward, memory_D = update_env(cur_data,cur_env,action, memory_D, cur_state)

        episode_reward += reward
        episode_step += 1
        action_count += 1

        if fit_count % 6 == 0:
            model = model_fit(model, model2, memory_D)
        if fit_count % 10000 == 0:
            model2.set_weights(model.get_weights())
#                 model2 = load_model('best_model(1).h5')
#                 model2.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
            print('\nmodel2 updated', j)
            print(round(acc_reward/action_count/(failed+1),2),' / ', round(best_acc_reward,2))
            print(acc_step,'번 시뮬레이션 결과')

            print('평균 개선 step 수 : ', round(advanced_step/acc_step,2))
            print('명령을 따랐는데 실패한 횟수 : ', failed)
            print('평균 Reward : ', round(acc_reward/action_count,2))
            print('exploration rate : ', exploration_rate)

            if acc_reward/action_count/(failed+1) >= best_acc_reward:
                print('best model saved')
                model.save('best_model(2).h5')

                best_acc_reward = acc_reward/action_count/(failed+1)
            acc_reward = 0
            acc_step=0
            advanced_step = 0
            succeeded = 0
            failed = 0
            action_count = 0
#         print('episode_reward : ', round(episode_reward,2))
#         print('스탭 개선 수 : ', cur_step - episode_step,'\n')
    advanced_step += cur_step - episode_step
    acc_reward += episode_reward
    exploration_rate = 0.05-0.05*fit_count/10**6




#             with open('memory_D(1).pickle', 'wb') as f:
#                 pickle.dump(memory_D, f)
#             if best_advanced_step > bestofbest_step:
#                 bestofbest_step = best_advanced_step
#                 model.save('bestofbest_model'+str(round(bestofbest_step,1))+'.h5')
#                 print('best of best model updated\n')
        


model2 updated 2622
0.0  /  0
2623 번 시뮬레이션 결과
평균 개선 step 수 :  4.9
명령을 따랐는데 실패한 횟수 :  106
평균 Reward :  0.39
exploration rate :  0.0495002
best model saved

model2 updated 5441
0.02  /  0.0
2819 번 시뮬레이션 결과
평균 개선 step 수 :  5.1
명령을 따랐는데 실패한 횟수 :  17
평균 Reward :  0.44
exploration rate :  0.049000150000000006
best model saved

model2 updated 8170
0.03  /  0.02
2729 번 시뮬레이션 결과
평균 개선 step 수 :  5.04
명령을 따랐는데 실패한 횟수 :  12
평균 Reward :  0.43
exploration rate :  0.048500100000000004
best model saved

model2 updated 10909
0.07  /  0.03
2739 번 시뮬레이션 결과
평균 개선 step 수 :  5.16
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.43
exploration rate :  0.048000100000000004
best model saved

model2 updated 13734
0.02  /  0.07
2825 번 시뮬레이션 결과
평균 개선 step 수 :  5.29
명령을 따랐는데 실패한 횟수 :  19
평균 Reward :  0.45
exploration rate :  0.0475001

model2 updated 16696
0.03  /  0.07
2962 번 시뮬레이션 결과
평균 개선 step 수 :  5.35
명령을 따랐는데 실패한 횟수 :  15
평균 Reward :  0.47
exploration rate :  0.04700005

model2 updated 19297
0.03  /  0.07
2601 번 시뮬레이션 결


model2 updated 135396
0.05  /  0.4
1951 번 시뮬레이션 결과
평균 개선 step 수 :  3.52
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.32
exploration rate :  0.02200015

model2 updated 137698
0.02  /  0.4
2302 번 시뮬레이션 결과
평균 개선 step 수 :  4.38
명령을 따랐는데 실패한 횟수 :  17
평균 Reward :  0.38
exploration rate :  0.021500050000000003

model2 updated 140116
0.06  /  0.4
2418 번 시뮬레이션 결과
평균 개선 step 수 :  4.44
명령을 따랐는데 실패한 횟수 :  6
평균 Reward :  0.4
exploration rate :  0.0210002

model2 updated 142400
0.03  /  0.4
2284 번 시뮬레이션 결과
평균 개선 step 수 :  4.25
명령을 따랐는데 실패한 횟수 :  14
평균 Reward :  0.38
exploration rate :  0.02050025

model2 updated 145239
0.12  /  0.4
2839 번 시뮬레이션 결과
평균 개선 step 수 :  5.1
명령을 따랐는데 실패한 횟수 :  3
평균 Reward :  0.47
exploration rate :  0.0200001

model2 updated 147830
0.06  /  0.4
2591 번 시뮬레이션 결과
평균 개선 step 수 :  4.82
명령을 따랐는데 실패한 횟수 :  6
평균 Reward :  0.43
exploration rate :  0.019500100000000003

model2 updated 149926
0.07  /  0.4
2096 번 시뮬레이션 결과
평균 개선 step 수 :  3.83
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.35
exploration 


model2 updated 246478
0.03  /  0.41
1775 번 시뮬레이션 결과
평균 개선 step 수 :  2.94
명령을 따랐는데 실패한 횟수 :  8
평균 Reward :  0.3
exploration rate :  -0.0049999000000000016

model2 updated 248189
0.1  /  0.41
1711 번 시뮬레이션 결과
평균 개선 step 수 :  2.91
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.29
exploration rate :  -0.005499900000000002

model2 updated 250029
0.05  /  0.41
1840 번 시뮬레이션 결과
평균 개선 step 수 :  3.06
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.31
exploration rate :  -0.005999950000000004

model2 updated 251749
0.07  /  0.41
1720 번 시뮬레이션 결과
평균 개선 step 수 :  2.76
명령을 따랐는데 실패한 횟수 :  3
평균 Reward :  0.29
exploration rate :  -0.006499649999999996

model2 updated 253492
0.01  /  0.41
1743 번 시뮬레이션 결과
평균 개선 step 수 :  2.84
명령을 따랐는데 실패한 횟수 :  36
평균 Reward :  0.29
exploration rate :  -0.006999950000000005

model2 updated 255383
0.05  /  0.41
1891 번 시뮬레이션 결과
평균 개선 step 수 :  3.22
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.32
exploration rate :  -0.007499850000000002

model2 updated 257260
0.05  /  0.41
1877 번 시뮬레이션 결과
평균 개선 step 수 :  3


model2 updated 354162
0.04  /  0.41
2515 번 시뮬레이션 결과
평균 개선 step 수 :  4.71
명령을 따랐는데 실패한 횟수 :  9
평균 Reward :  0.43
exploration rate :  -0.03199990000000001

model2 updated 356435
0.13  /  0.41
2273 번 시뮬레이션 결과
평균 개선 step 수 :  4.24
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.39
exploration rate :  -0.032499799999999995

model2 updated 358684
0.13  /  0.41
2249 번 시뮬레이션 결과
평균 개선 step 수 :  4.21
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.39
exploration rate :  -0.032999799999999996

model2 updated 360898
0.08  /  0.41
2214 번 시뮬레이션 결과
평균 개선 step 수 :  4.0
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.38
exploration rate :  -0.033499799999999996

model2 updated 363060
0.37  /  0.41
2162 번 시뮬레이션 결과
평균 개선 step 수 :  4.06
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.37
exploration rate :  -0.033999749999999995

model2 updated 365370
0.07  /  0.41
2310 번 시뮬레이션 결과
평균 개선 step 수 :  4.23
명령을 따랐는데 실패한 횟수 :  5
평균 Reward :  0.39
exploration rate :  -0.0344998

model2 updated 367775
0.03  /  0.41
2405 번 시뮬레이션 결과
평균 개선 step 수 :  4.33
명령을 따랐는데 


model2 updated 463506
0.02  /  0.41
1872 번 시뮬레이션 결과
평균 개선 step 수 :  3.29
명령을 따랐는데 실패한 횟수 :  16
평균 Reward :  0.32
exploration rate :  -0.059499750000000004

model2 updated 465449
0.04  /  0.41
1943 번 시뮬레이션 결과
평균 개선 step 수 :  3.48
명령을 따랐는데 실패한 횟수 :  7
평균 Reward :  0.33
exploration rate :  -0.05999985000000001

model2 updated 467321
0.11  /  0.41
1872 번 시뮬레이션 결과
평균 개선 step 수 :  3.42
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.32
exploration rate :  -0.060499800000000006

model2 updated 469109
0.15  /  0.41
1788 번 시뮬레이션 결과
평균 개선 step 수 :  3.14
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.31
exploration rate :  -0.06099985000000001

model2 updated 470943
0.08  /  0.41
1834 번 시뮬레이션 결과
평균 개선 step 수 :  3.08
명령을 따랐는데 실패한 횟수 :  3
평균 Reward :  0.31
exploration rate :  -0.06149995000000001

model2 updated 473199
0.19  /  0.41
2256 번 시뮬레이션 결과
평균 개선 step 수 :  4.14
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.39
exploration rate :  -0.061999799999999994

model2 updated 475499
0.1  /  0.41
2300 번 시뮬레이션 결과
평균 개선 step 수 :  4.27


model2 updated 579658
0.01  /  0.41
2007 번 시뮬레이션 결과
평균 개선 step 수 :  3.65
명령을 따랐는데 실패한 횟수 :  57
평균 Reward :  0.33
exploration rate :  -0.0874998

model2 updated 581861
0.08  /  0.41
2203 번 시뮬레이션 결과
평균 개선 step 수 :  4.05
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.38
exploration rate :  -0.0879997

model2 updated 584262
0.1  /  0.41
2401 번 시뮬레이션 결과
평균 개선 step 수 :  4.38
명령을 따랐는데 실패한 횟수 :  3
평균 Reward :  0.41
exploration rate :  -0.0884998

model2 updated 586303
0.09  /  0.41
2041 번 시뮬레이션 결과
평균 개선 step 수 :  3.81
명령을 따랐는데 실패한 횟수 :  3
평균 Reward :  0.35
exploration rate :  -0.08899975

model2 updated 588286
0.07  /  0.41
1983 번 시뮬레이션 결과
평균 개선 step 수 :  3.56
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.34
exploration rate :  -0.08949975

model2 updated 590541
0.02  /  0.41
2255 번 시뮬레이션 결과
평균 개선 step 수 :  4.12
명령을 따랐는데 실패한 횟수 :  24
평균 Reward :  0.38
exploration rate :  -0.08999975

model2 updated 592779
0.02  /  0.41
2238 번 시뮬레이션 결과
평균 개선 step 수 :  4.07
명령을 따랐는데 실패한 횟수 :  22
평균 Reward :  0.38
exploration rate :


model2 updated 705100
0.23  /  0.49
2617 번 시뮬레이션 결과
평균 개선 step 수 :  4.8
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.45
exploration rate :  -0.11499990000000003

model2 updated 707543
0.08  /  0.49
2443 번 시뮬레이션 결과
평균 개선 step 수 :  4.51
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.42
exploration rate :  -0.11549980000000003

model2 updated 709711
0.19  /  0.49
2168 번 시뮬레이션 결과
평균 개선 step 수 :  3.99
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.37
exploration rate :  -0.11599995

model2 updated 711841
0.37  /  0.49
2130 번 시뮬레이션 결과
평균 개선 step 수 :  4.0
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.37
exploration rate :  -0.11649995

model2 updated 714381
0.09  /  0.49
2540 번 시뮬레이션 결과
평균 개선 step 수 :  4.65
명령을 따랐는데 실패한 횟수 :  4
평균 Reward :  0.44
exploration rate :  -0.11699985

model2 updated 716458
0.18  /  0.49
2077 번 시뮬레이션 결과
평균 개선 step 수 :  3.79
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.36
exploration rate :  -0.1174997

model2 updated 718656
0.19  /  0.49
2198 번 시뮬레이션 결과
평균 개선 step 수 :  3.99
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.38
ex


model2 updated 832916
0.43  /  0.49
2505 번 시뮬레이션 결과
평균 개선 step 수 :  4.54
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.43
exploration rate :  -0.14299984999999998

model2 updated 835134
0.13  /  0.49
2218 번 시뮬레이션 결과
평균 개선 step 수 :  4.11
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.38
exploration rate :  -0.14349974999999998

model2 updated 837251
0.37  /  0.49
2117 번 시뮬레이션 결과
평균 개선 step 수 :  3.86
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.37
exploration rate :  -0.14399974999999998

model2 updated 839597
0.4  /  0.49
2346 번 시뮬레이션 결과
평균 개선 step 수 :  4.48
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.4
exploration rate :  -0.14449984999999999

model2 updated 841830
0.13  /  0.49
2233 번 시뮬레이션 결과
평균 개선 step 수 :  4.03
명령을 따랐는데 실패한 횟수 :  2
평균 Reward :  0.38
exploration rate :  -0.14499984999999999

model2 updated 843897
0.35  /  0.49
2067 번 시뮬레이션 결과
평균 개선 step 수 :  3.85
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.35
exploration rate :  -0.1454997

model2 updated 846053
0.37  /  0.49
2156 번 시뮬레이션 결과
평균 개선 step 수 :  3.87
명령을 따랐는데 실패한 횟


model2 updated 956551
0.37  /  0.49
2136 번 시뮬레이션 결과
평균 개선 step 수 :  3.87
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.37
exploration rate :  -0.17049970000000003

model2 updated 958572
0.17  /  0.49
2021 번 시뮬레이션 결과
평균 개선 step 수 :  3.61
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.35
exploration rate :  -0.17099990000000004

model2 updated 960730
0.37  /  0.49
2158 번 시뮬레이션 결과
평균 개선 step 수 :  3.9
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.37
exploration rate :  -0.17149995

model2 updated 962955
0.19  /  0.49
2225 번 시뮬레이션 결과
평균 개선 step 수 :  4.09
명령을 따랐는데 실패한 횟수 :  1
평균 Reward :  0.38
exploration rate :  -0.17199985

model2 updated 965171
0.38  /  0.49
2216 번 시뮬레이션 결과
평균 개선 step 수 :  4.19
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.38
exploration rate :  -0.17249990000000004

model2 updated 967391
0.38  /  0.49
2220 번 시뮬레이션 결과
평균 개선 step 수 :  4.17
명령을 따랐는데 실패한 횟수 :  0
평균 Reward :  0.38
exploration rate :  -0.17299990000000004

model2 updated 969568
0.37  /  0.49
2177 번 시뮬레이션 결과
평균 개선 step 수 :  4.04
명령을 따랐는데 실패한 횟수 :  0


## 검증

In [21]:
############## 저장된 모델 불러오는 부분########
from tensorflow.keras.models import load_model
model = load_model('best_model(2).h5')
model.compile(optimizer = 'rmsprop', loss='mse', metrics=['mae'])
###############################################

advanced_step=0
acc_reward=0
failed=0
acc_origin_step = 0
acc_episode_step=0
for k in range(len(test_df_list)):
    if k%1000==0:
        print(k,end=' ')
    cur_data = select_load(test_df_list,k)
    acc_origin_step += cur_data[1]
    cur_env = init_env(cur_data)
    terminal = False
    # cur_state: [[현재 z 거리, 바로 전 z 거리, 현재 SFR, 바로 전 SFR, 기울기]]
    cur_state = np.expand_dims(np.asarray([cur_data[0].iloc[0,1], cur_data[0].iloc[0,1], cur_env[1], cur_env[1],0]),axis=0)
    reward = 0
    cur_step = 0
    episode_reward = 0
    episode_step = 0
    
    while terminal == False:
        action = best_decide(model, cur_state)
        cur_step += action
#             print(cur_step, end = ' ')
        past_state = cur_state
        cur_env, cur_state, reward, memory_D = update_env(cur_data,cur_env,action, memory_D, cur_state)
        episode_reward += reward
        episode_step += 1
#         print('episode_reward : ', round(episode_reward,2))
#         print('스탭 개선 수 : ', cur_step - episode_step,'\n')
    advanced_step += cur_step - episode_step
    acc_episode_step += episode_step
    acc_reward += episode_reward
print('\n모든 데이터 시뮬레이션 결과 정리')
print('평균 개선 step 수 : ', advanced_step/len(test_df_list))
print('개선율 : ', 100*acc_episode_step/acc_origin_step, '%')
print('실패율 : ', 100*failed/len(test_df_list),'%')
print('명령을 따랐는데 실패한 횟수 : ', failed)
print('평균 누적 Reward : ', round(acc_reward/len(test_df_list),2),'\n')


        
    

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 
모든 데이터 시뮬레이션 결과 정리
평균 개선 step 수 :  5.168292529422871
개선율 :  39.823229362101316 %
실패율 :  0.03776197369249166 %
명령을 따랐는데 실패한 횟수 :  6
평균 누적 Reward :  1.68 



## 아래는 데이터 전처리하였던 코드 (정리 안된 상태)

In [22]:
count=0
idx=[]
for i in range(len(df_list)):
    if str(type(df_list[i].loc[0,'CentSFR 0.7'])) != "<class 'numpy.float64'>":
        idx.append(i)
        count+=1
        print(i)
        print(df_list[i])
#         if len(df_list[i])<10:
#             print('뷁',i)
#         if len(df_list[i])<10:
#             count+=1
#             print(i, df_list[i]['CentSFR 0.7'].argmax())
#             print(df_list[i])
print(count)

0


In [23]:
df_list[idx[0]].index[10]

IndexError: list index out of range

In [ ]:
k=0
for i in idx:
    tmp_df = df_list[i-k]
    tmp_idx=[]
    for j in range(len(tmp_df)):
        
        if tmp_df.index[j]==0:
            tmp_idx.append(j)
    for j in range(len(tmp_idx)):
        if j != len(tmp_idx)-1:
            df_list.append(tmp_df.iloc[tmp_idx[j]:tmp_idx[j+1],:])
        else:
            df_list.append(tmp_df.iloc[tmp_idx[j]:,:])
    df_list.pop(i-k)
    k+=1

In [ ]:
i

In [ ]:
for i in range(len(df_list)):
    if df_list[i].index[0]!=0:
        print(i)
        print(df_list[i])

In [ ]:
df_list.pop(48503)

In [ ]:
for i in range(len(idx)):
    if len(df_list[idx[i]].loc[0,:]) != 2:
        print(len(df_list[idx[i]].loc[0,:]))

In [ ]:
len(df_list)

In [ ]:
df_list[4316].index[0]

In [ ]:
df_list[9313].iloc[2:,2].argmax()+2

In [ ]:
df_list[9313].iloc[2:]

In [ ]:
df_list[1062]['CentSFR 0.7'].iloc[2:].argmax()

In [ ]:
df_list[1062]['CentSFR 0.7'].iloc[:]

In [ ]:
len(memory_D)